In [29]:
import pandas as pd
import csv
import langid
import numpy as np
import requests
import json
import re
import string
import nltk
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import defaultdict

In [30]:
# download the Indonesian stopwords from NLTK
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('indonesian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\irfan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
# Load data
data = pd.read_csv('Bahan/04.IbuKotaNegara.csv', encoding='utf-8')

In [32]:
# Membuat kamus kata baku dan tidak baku
baku = defaultdict(set)
with open("Bahan/kata_dasar_kbbi.csv", "r", encoding="utf-8") as file:
    reader = csv.reader(file)
    next(reader)  # Skip header
    for row in reader:
        kata_baku = row[0]
        kategori = row[1]
        baku[kata_baku].add(kategori)

In [33]:
# Cek apakah 'case_folding' ada dalam dataframe
if 'text_tweet' in data.columns:
    # Ambil kolom 'case_folding' dari dataframe
    kolom_case_folding = data['text_tweet']
    
    # Fungsi untuk memisahkan hashtag
    def pisahkan_hashtag(kalimat):
        # Menggunakan regex untuk mencocokkan pola hashtag
        pola = r'#[A-Za-z0-9_]+'
        hasil = re.findall(pola, kalimat)
        
        # Mengganti hashtag dengan kata terpisah
        for hashtag in hasil:
            kata_terpisah = re.sub(r'([a-z])([A-Z])', r'\1 \2', hashtag)
            kalimat = kalimat.replace(hashtag, kata_terpisah)
        
        return kalimat

    # Memisahkan hashtag pada setiap kalimat dalam kolom 'case_folding'
    data['pisah_hashtag&kata'] = kolom_case_folding.apply(pisahkan_hashtag)
    
    # Cetak dataframe untuk memverifikasi hasilnya
    print(data)
else:
    print("Kolom 'case_folding' tidak ada dalam dataframe.")

     Unnamed: 0                                         text_tweet  \
0           626  @KoalisiNetizen Peningkatan akses transportasi...   
1           629  Seorang Warga negara haram Isr4el Avitan Shalo...   
2           847  Menteri PUPR Basuki Hadimuljono mengungkapkan ...   
3           514  Ternyata masalah lahan di Ibu Kota Negara (IKN...   
4           365  Sebuah langkah besar menuju masa depan yang le...   
..          ...                                                ...   
495         100  Perjalanan Berkelanjutan: Sistem Transportasi ...   
496         426  @abangbelneg Kami warga Bengkulu mendukung pem...   
497         178  @IKNnusantaraID Ibu kota nusantara akan menjad...   
498         444  @PolJokesID Karena Nusantara minim investor ib...   
499         416  @abangbelneg Semoga pembangunan IKN dilancarka...   

                                    pisah_hashtag&kata  
0    @KoalisiNetizen Peningkatan akses transportasi...  
1    Seorang Warga negara haram Isr4el Avitan

In [34]:
# Import corpus dari nltk
from nltk.tokenize import word_tokenize

# Fungsi untuk menghapus tanda baca menggunakan regex
def hapus_tanda_baca(kata):
    return re.sub(r'[^\w\s-]', '', kata)

# Fungsi untuk melakukan preprocessing pada kolom teks
def preprocess_text(text):

    # Menghapus karakter non-ASCII
    text = remove_non_ascii(text)

    # Remove URLs, mentions, and hashtags
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'\@\w+|\#', '', text)

    # Tokenisasi kalimat menjadi kata-kata
    kata_kata = word_tokenize(text)

    # Menghapus tanda baca dari setiap kata
    kata_kata_tanpa_tanda_baca = [hapus_tanda_baca(kata) for kata in kata_kata]

    # Menggabungkan kata-kata kembali menjadi kalimat
    kalimat_tanpa_tanda_baca = ' '.join(kata_kata_tanpa_tanda_baca)

    # Menggantikan tanda baca dengan spasi tunggal menggunakan regex
    kalimat_final = re.sub(r'(?<=[^\w\s-])', ' ', kalimat_tanpa_tanda_baca)

    # Menggantikan multiple spasi dengan satu spasi menggunakan regex
    kalimat_tanpa_spasi_ganda = re.sub(r'\s+', ' ', kalimat_final)

    # Remove special characters and digits
    kalimat_tanpa_tanda_baca_dan_digit = re.sub(r'[^\w\s-]', '', kalimat_tanpa_spasi_ganda)
    kalimat_tanpa_digit = re.sub(r'\d+', '', kalimat_tanpa_tanda_baca_dan_digit)

    # Convert to lowercase
    kalimat_final = kalimat_tanpa_digit.lower()

    # Remove whitespace leading & trailing
    kalimat_final = kalimat_final.strip()

    # Remove single char
    kalimat_final = re.sub(r"\b[a-zA-Z]\b", "", kalimat_final)

    return kalimat_final

def remove_non_ascii(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text) if unicodedata.category(c) != 'Mn')

# Melakukan preprocessing pada kolom teks
data['text_cleaning'] = data['pisah_hashtag&kata'].apply(preprocess_text)

In [35]:
data

,Unnamed: 0,text_tweet,pisah_hashtag&kata,text_cleaning
0,626,@KoalisiNetizen Peningkatan akses transportasi...,@KoalisiNetizen Peningkatan akses transportasi...,peningkatan akses transportasi akan mendorong ...
1,629,Seorang Warga negara haram Isr4el Avitan Shalo...,Seorang Warga negara haram Isr4el Avitan Shalo...,seorang warga negara haram isrel avitan shalom...
2,847,Menteri PUPR Basuki Hadimuljono mengungkapkan ...,Menteri PUPR Basuki Hadimuljono mengungkapkan ...,menteri pupr basuki hadimuljono mengungkapkan ...
3,514,Ternyata masalah lahan di Ibu Kota Negara (IKN...,Ternyata masalah lahan di Ibu Kota Negara (IKN...,ternyata masalah lahan di ibu kota negara ikn ...
4,365,Sebuah langkah besar menuju masa depan yang le...,Sebuah langkah besar menuju masa depan yang le...,sebuah langkah besar menuju masa depan yang le...
...,...,...,...,...
495,100,Perjalanan Berkelanjutan: Sistem Transportasi ...,Perjalanan Berkelanjutan: Sistem Transportasi ...,perjalanan berkelanjutan sistem transportasi c...
496,426,@abangbelneg Kami warga Bengkulu mendukung pem...,@abangbelneg Kami warga Bengkulu mendukung pem...,kami warga bengkulu mendukung pemindahan ibu k...
497,178,@IKNnusantaraID Ibu kota nusantara akan menjad...,@IKNnusantaraID Ibu kota nusantara akan menjad...,ibu kota nusantara akan menjadi simbol kebangk...
498,444,@PolJokesID Karena Nusantara minim investor ib...,@PolJokesID Karena Nusantara minim investor ib...,karena nusantara minim investor ibu kota negar...


In [36]:

# Membuat list untuk menyimpan kata baku dan tidak baku dalam setiap tweet
data_kata_baku_dan_tidak_baku = []

In [37]:
# Melakukan identifikasi kata baku dan tidak baku dalam setiap tweet
for i, tweet in data.iterrows():
    text = tweet["text_cleaning"]
    tokens = text.split()
    kata_baku = set()  # Menggunakan set untuk menghindari duplikasi
    kata_tidak_baku = set()  # Menggunakan set untuk menghindari duplikasi
    for token in tokens:
        if token.lower() in baku:
            kata_baku.add(token)
        else:
            kata_tidak_baku.add(token)
    data_kata_baku_dan_tidak_baku.append({
        "Kata Baku": " ".join(kata_baku),
        "Kata Tidak Baku": " ".join(kata_tidak_baku)
    })

# Membuat DataFrame dari kata baku dan tidak baku dalam setiap tweet
df_kata_baku_dan_tidak_baku = pd.DataFrame(data_kata_baku_dan_tidak_baku)

# Menyimpan DataFrame ke dalam file CSV
df_kata_baku_dan_tidak_baku.to_csv("Coba/daftar kata baku dan tidak baku 02.csv", index=False, header=True)

In [38]:
# Membuat list untuk menyimpan data yang tidak baku
data_tidak_baku = []

In [39]:
from collections import Counter

# Melakukan identifikasi kata tidak baku
data_tidak_baku = []
for i, tweet in data.iterrows():
    text = tweet["text_cleaning"]
    tokens = text.split()
    for j, token in enumerate(tokens):
        if token.lower() not in baku:
            data_tidak_baku.append({
                "text": token,
                "edit": "",
            })

# Menghitung kata-kata tidak baku yang sering muncul
counter = Counter(token["text"].lower() for token in data_tidak_baku)
kata_tidak_baku_serimg_muncul = counter.most_common()  # Mengambil semua kata yang sering muncul

# Menampilkan kata-kata tidak baku yang sering muncul
print("Kata-kata tidak baku yang sering muncul:")
for kata, frekuensi in kata_tidak_baku_serimg_muncul:
    print(f"{kata}: {frekuensi}")

# Membuat DataFrame dari kata-kata tidak baku yang sering muncul
df_kata_tidak_baku = pd.DataFrame(kata_tidak_baku_serimg_muncul, columns=["Kata Tidak Baku", "Frekuensi"])

# Menyimpan DataFrame ke dalam file CSV
df_kata_tidak_baku.to_csv("Coba/kata_tidak_baku.csv", index=False)

Kata-kata tidak baku yang sering muncul:
ikn: 220
iknnusantara: 107
pemindahan: 76
jakarta: 71
pembangunan: 64
kalimantan: 60
mendukung: 46
menjadi: 42
kaltim: 34
yg: 30
sebagai: 26
jokowi: 25
pemerintah: 22
sebuah: 18
menuju: 17
ibukota: 17
peningkatan: 16
dukungan: 16
tersebut: 16
lingkungan: 16
dulu: 15
-: 14
terhadap: 14
asn: 14
membangun: 13
semoga: 13
secara: 13
pertumbuhan: 12
memberikan: 12
meningkatkan: 12
pendidikan: 12
semakin: 11
ri: 11
dkj: 11
uu: 11
dki: 10
prabowo: 10
otorita: 10
berkesinambungan: 9
mendatang: 9
amp: 9
setelah: 9
bersama: 9
berkelanjutan: 9
ga: 9
udah: 9
peluang: 9
sd: 9
pupr: 8
berstatus: 8
city: 8
ahy: 8
smart: 8
sepaku: 8
berkembang: 8
vvip: 8
memiliki: 7
ppu: 7
new: 7
sydney: 7
mengawal: 7
memperkuat: 7
membuka: 7
mewujudkan: 7
sebanyak: 7
lapangan: 7
memastikan: 7
sampe: 7
gak: 7
bagian: 6
kotanya: 6
dilakukan: 6
melalui: 6
jaman: 6
penduduk: 6
memajukan: 6
keberlanjutan: 6
kehidupan: 6
polda: 6
kamtibmas: 6
pengamanan: 6
hapal: 6
mendorong: 5
mengu

In [40]:
# Menyimpan data yang tidak baku ke dalam file CSV baru
df_tidak_baku = pd.DataFrame(data_tidak_baku, columns=["text"])
df_tidak_baku.to_csv("Coba/kumpulan_kata_tidak_baku.txt", sep="\t", index=False)